In [1]:
import torch.backends.cudnn as cudnn
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from models.modifiedresnet import ModifiedResNet18 
from models.dropoutresnet import DropoutResNet18
import torch
import torchvision
import torch.nn.parallel
import torchvision.transforms as transforms
from collections import OrderedDict
import torch.nn as nn
import torch.optim as optim

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = DropoutResNet18(0.4)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
checkpoint = torch.load('./checkpoint/best_epoch_3.pth', map_location=device)
net.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [3]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
testset = torchvision.datasets.CIFAR10(
    root='./deep-learning-mini-project-spring-24-nyu/cifar-10-python', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified


In [4]:
# switch to eval mode
net.eval()
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(testloader):
    
        # start test epoch
        inputs, targets = inputs.to(device), targets.to(device)

        # forward pass
        outputs = net(inputs)
        
        # calculate loss and accuracy
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

Loss: 0.000 | Acc: 96.100% (9610/10000)
